<a href="https://colab.research.google.com/github/jumafernandez/data-science-II/blob/main/Stacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Métodos de ensamble: Stacking

En esta notebook implementamos un pequeño ejemplo de utilización de métodos de ensamble mediante la estrategia de _stacking_, donde hay un conjunto de modelos independientes que aprenden de los datos y luego un _meta-modelo_ que aprende a partir de las predicciones de los modelos independientes, lo cual se ilustra en el siguiente esquema:

<img src="https://editor.analyticsvidhya.com/uploads/39725Stacking.png" alt="Esquema stacking general" width="500">


Cargamos las librerías necesarias:

In [7]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score

Cargamos el dataset a utilizar, en nuestro caso el de cancer de mama y lo separamos en train y test:

In [15]:
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape

(455, 30)

Entrenamos modelos base por separado y verificamos el accuracy:

In [9]:
# Modelos base
clf1 = DecisionTreeClassifier(random_state=42)
clf2 = SVC(probability=True, random_state=42)  # Es necesario probability=True para stacking
clf3 = KNeighborsClassifier(n_neighbors=3)

# Entrenamiento y evaluación individual de cada modelo
clf1.fit(X_train, y_train)
y_pred1 = clf1.predict(X_test)
print(f"Accuracy Decision Tree: {accuracy_score(y_test, y_pred1):.2f}")

clf2.fit(X_train, y_train)
y_pred2 = clf2.predict(X_test)
print(f"Accuracy SVM: {accuracy_score(y_test, y_pred2):.2f}")

clf3.fit(X_train, y_train)
y_pred3 = clf3.predict(X_test)
print(f"Accuracy KNN: {accuracy_score(y_test, y_pred3):.2f}")

Accuracy Decision Tree: 0.95
Accuracy SVM: 0.95
Accuracy KNN: 0.93


Ahora entrenamos el ensamble a través de la estrategia de stacking:

In [10]:
# Aplicación de Stacking
estimators = [
    ('decision_tree', DecisionTreeClassifier(random_state=42)),
    ('svm', SVC(probability=True, random_state=42)),
    ('knn', KNeighborsClassifier(n_neighbors=3))
    ]

# Meta-modelo: Regresión Logística
stacking_clf = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(random_state=42)
)

# Entrenamiento del modelo Stacking
stacking_clf.fit(X_train, y_train)
y_pred_stack = stacking_clf.predict(X_test)

# Evaluación del modelo Stacking
print(f"Accuracy Stacking: {accuracy_score(y_test, y_pred_stack):.2f}")


Accuracy Stacking: 0.97


Ahora vamos a intentar entender que es lo que hace la estrategia de _stacking_.

Primero es posible armar un dataframe con las estimaciones de los modelos independientes (árbol de clasificación, máquina vector soporte y vecinos más cercanos:

In [20]:
import pandas as pd

# Acceder a las predicciones de los estimadores base para entrenar el meta-modelo
predicciones_estimadores_base = stacking_clf.transform(X_train)

# Obtener los nombres de los estimadores base
estimadores_base_nombres = [est.__class__.__name__ for est in stacking_clf.estimators_]

# Crear un DataFrame con las predicciones de los estimadores base
df_predicciones = pd.DataFrame(predicciones_estimadores_base, columns=estimadores_base_nombres)

# Mostrar las primeras filas del DataFrame
print(df_predicciones.head())

   DecisionTreeClassifier       SVC  KNeighborsClassifier
0                     1.0  0.972563                   1.0
1                     0.0  0.000725                   0.0
2                     1.0  0.976237                   1.0
3                     1.0  0.964614                   1.0
4                     1.0  0.976413                   1.0


También podemos acceder a los coeficientes de la regresión logística que funciona como __meta-modelo__:

In [22]:
# Verificar los nombres de los estimadores
estimadores_base_nombres = [est.__class__.__name__ for est in stacking_clf.estimators_]

# Acceder a los coeficientes de la regresión logística
log_reg = stacking_clf.final_estimator_
coeficientes = log_reg.coef_
intercepto = log_reg.intercept_

# Mostrar los coeficientes asociados a cada estimador
for est_name, coef in zip(estimadores_base_nombres, coeficientes[0]):
    print(f"Coeficiente para {est_name}: {coef}")

# Mostrar intercepto
print("Intercepto de la regresión logística:", intercepto)

Coeficiente para DecisionTreeClassifier: 2.4439015481592397
Coeficiente para SVC: 2.1907700553846414
Coeficiente para KNeighborsClassifier: 2.5838222714630095
Intercepto de la regresión logística: [-3.71698167]


Por último, podemos utilizar los valores del dataset y los coeficientes del modelo de regresión logística para simular la clasificación que realiza el clasificador:

In [25]:
import numpy as np

# Aplanar el coeficiente, si es necesario
coeficientes = log_reg.coef_[0]  # Asegúrate de que esto sea 1D

# Usar la fórmula de la regresión logística: z = X * coef + intercepto
z = np.dot(df_predicciones.values, coeficientes) + intercepto

# Aplicar la función sigmoide para obtener probabilidades
predicciones_meta_modelo = 1 / (1 + np.exp(-z))

# Agregar las predicciones al DataFrame
df_predicciones['prediccion_meta_modelo'] = predicciones_meta_modelo

   DecisionTreeClassifier       SVC  KNeighborsClassifier  \
0                     1.0  0.972563                   1.0   
1                     0.0  0.000725                   0.0   
2                     1.0  0.976237                   1.0   
3                     1.0  0.964614                   1.0   
4                     1.0  0.976413                   1.0   

   prediccion_meta_modelo  
0                0.968974  
1                0.023767  
2                0.969215  
3                0.968446  
4                0.969226  


Ahora podemos ver en prediccion_meta_modelo la probabilidad de pertenecer a la clase positiva:

In [26]:
print(df_predicciones.head())

   DecisionTreeClassifier       SVC  KNeighborsClassifier  \
0                     1.0  0.972563                   1.0   
1                     0.0  0.000725                   0.0   
2                     1.0  0.976237                   1.0   
3                     1.0  0.964614                   1.0   
4                     1.0  0.976413                   1.0   

   prediccion_meta_modelo  
0                0.968974  
1                0.023767  
2                0.969215  
3                0.968446  
4                0.969226  


Y podemos transformar eso en una predicción de clase:

In [27]:
# Agregar una nueva columna 'clasificacion' basada en las reglas proporcionadas
df_predicciones['clasificacion'] = np.where(df_predicciones['prediccion_meta_modelo'] >= 0.5, 'Positiva', 'Negativa')

# Mostrar las primeras filas del DataFrame con la nueva columna de clasificación
print(df_predicciones.head())


   DecisionTreeClassifier       SVC  KNeighborsClassifier  \
0                     1.0  0.972563                   1.0   
1                     0.0  0.000725                   0.0   
2                     1.0  0.976237                   1.0   
3                     1.0  0.964614                   1.0   
4                     1.0  0.976413                   1.0   

   prediccion_meta_modelo clasificacion  
0                0.968974      Positiva  
1                0.023767      Negativa  
2                0.969215      Positiva  
3                0.968446      Positiva  
4                0.969226      Positiva  
